In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import zipfile
import io
import geopandas as gpd
import s3fs
import zarr
import dask 
import dask.array
import xarray as xr
import pandas as pd
from shapely.geometry import Point
from shapely.prepared import prep
import rioxarray




In [ ]:
Start_date = 2013
End_date = 2023

# Task 1 : Download the Texas shape file.

In [ ]:
url = "https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_5m.zip"
response = requests.get(url)

if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall("path_to_extract")  # م

# بارگذاری فایل‌های استخراج شده
shapefile_path = "path_to_extract/cb_2023_us_state_5m.shp"
all_states = gpd.read_file(shapefile_path)
texas_shape = all_states[all_states['NAME'] == 'Texas'].to_crs('EPSG:4326')


In [ ]:
# fig = plt.figure(figsize=(12, 10))
# texas_shape.plot(edgecolor='blue', facecolor='blue', alpha=0.3)
# plt.grid(True)
# plt.title("Texas Shape from US Census Bureau")
# plt.show()

In [ ]:

base_url = 'noaa-nws-aorc-v1-1-1km' 
dataset_years = list(range(Start_date, End_date))
s3_out = s3fs.S3FileSystem(anon=True)
#store = s3fs.S3Map(root=f"{base_url}/{year}.zarr", s3=s3_out, check=False)

store = [s3fs.S3Map(
            root=f"noaa-nws-aorc-v1-1-1km/{dataset_year}.zarr",
            s3=s3fs.S3FileSystem(anon=True),
            check=False
        ) for dataset_year in dataset_years]


In [ ]:
base_url = 'noaa-nws-aorc-v1-1-1km'
s3_out = s3fs.S3FileSystem(anon=True)

texas_bbox = texas_shape.bounds
minx, miny, maxx, maxy = texas_bbox.iloc[0]

print(f"Texas bounding box: {minx}, {miny}, {maxx}, {maxy}")

available_years = []
for year in range(Start_date, End_date):
    if s3_out.exists(f"{base_url}/{year}.zarr"):
        available_years.append(year)

print(f"Available years: {available_years}")
texas_datasets = []

for year in available_years:
    print(f"\nProcessing year: {year}")
    
    try:
        store = s3fs.S3Map(root=f"{base_url}/{year}.zarr", s3=s3_out, check=False)
        ds = xr.open_dataset(store, engine='zarr', chunks={'time': 720}) 
        lat_values = ds.latitude.values
        if lat_values[0] > lat_values[-1]:
            ds_texas_bbox = ds.sel(latitude=slice(maxy + 1, miny - 1))

        ds_texas_bbox = ds.sel(
            longitude=slice(minx - 1, maxx + 1),
            latitude=slice(miny - 1, maxy + 1)
        )

        
        ds_texas_bbox = ds_texas_bbox.rio.write_crs("EPSG:4326")
        ds_texas = ds_texas_bbox.rio.clip(texas_shape.geometry, drop=True)
        texas_datasets.append(ds_texas)  
    except Exception as e:
        print(f"Error processing year {year}: {e}")

In [ ]:
ds_texas_bbox = ds_texas_bbox.rio.write_crs("EPSG:4326", inplace=True)
if texas_shape.crs != ds_texas_bbox.rio.crs:
    texas_shape = texas_shape.to_crs(ds_texas_bbox.rio.crs)

ds_texas = ds_texas_bbox.rio.clip(texas_shape.geometry, drop=True)
print("After clip:", ds_texas)


In [ ]:
ds_texas = ds_texas.chunk({'latitude': 200, 'longitude': 200, 'time': 720})

In [ ]:
# Ensure latitude is ascending after clip
if ds_texas.latitude.values[0] > ds_texas.latitude.values[-1]:
    ds_texas = ds_texas.sortby('latitude')


In [ ]:
ds_texas = xr.concat(texas_datasets, dim='time')

In [ ]:
# single_timestep = ds_texas['TMP_2maboveground'].isel(time=0)
# fig, ax = plt.subplots(1, 1, figsize=(12, 10))
# ax.set_title(f'Plot WITHOUT Cartopy Features', fontsize=16)
# single_timestep.plot(ax=ax, cmap='coolwarm', add_colorbar=False)
# texas_shape.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

# plt.show()

## Task 2: Label grids by numbering them from 1 to ~700,000. You should start from top left of Texas to bottom right.

In [ ]:

with dask.config.set(array_slicing={"split_large_chunks": False}):
    stacked_ds = ds_texas.stack(grid=("latitude", "longitude"))

num_grids = stacked_ds.grid.size
print(f"Total number of grids (before masking): {num_grids}")

In [ ]:
from shapely import points

nrows, ncols = len(ds_texas.latitude), len(ds_texas.longitude)
grid_labels = np.arange(1, num_grids + 1)
labels_2d = grid_labels.reshape((nrows, ncols))

# Fix latitude orientation
lat_ascending = (ds_texas.latitude.values[1] > ds_texas.latitude.values[0])
if lat_ascending:
    labels_2d = np.flipud(labels_2d)

label_da = xr.DataArray(
    labels_2d,
    coords={"latitude": ds_texas.latitude, "longitude": ds_texas.longitude},
    dims=["latitude", "longitude"]
)

# Use union_all instead of unary_union
texas_geom = texas_shape.union_all()
texas_geom_prep = prep(texas_geom)

# Vectorized shapely evaluation (MUCH faster)
lon, lat = np.meshgrid(ds_texas.longitude.values, ds_texas.latitude.values)
pts = points(lon.ravel(), lat.ravel())                 # Vectorized point creation
mask_flat = np.array([texas_geom_prep.contains(p) for p in pts])
mask = mask_flat.reshape(lon.shape)


In [ ]:
masked_labels = label_da.where(mask)
num_valid_grids = int(masked_labels.notnull().sum().values)
print(f"Number of grid cells inside Texas (after masking): {num_valid_grids}")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.set_title('Task 2: Labeled Grids Inside Texas', fontsize=16)
plot = masked_labels.plot(ax=ax, cmap='viridis', add_colorbar=True)
plot.colorbar.set_label('Grid Cell ID', size=12)
texas_shape.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.grid(True)
plt.show()

### Verification that the grids are labelled as innstructed

In [ ]:
first_label_val = masked_labels.min().item()
last_label_val = masked_labels.max().item()

In [ ]:
first_label_cell = masked_labels.where(masked_labels == first_label_val, drop=True)
lat_1, lon_1 = first_label_cell.latitude.values[0], first_label_cell.longitude.values[0]

last_label_cell = masked_labels.where(masked_labels == last_label_val, drop=True)
lat_last, lon_last = last_label_cell.latitude.values[0], last_label_cell.longitude.values[0]

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 10))
# plot = masked_labels.plot(ax=ax, cmap='viridis', add_colorbar=True)
# plot.colorbar.set_label('Grid Cell ID', size=12)
# texas_shape.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)
# ax.scatter(lon_1, lat_1, color='cyan', marker='*', s=300, edgecolor='black', linewidth=1.5,
#            label=f'Min Label (Top-Left): {int(first_label_val)}')
# ax.scatter(lon_last, lat_last, color='magenta', marker='X', s=300, edgecolor='white', linewidth=1.5,
#            label=f'Max Label (Bottom-Right): {int(last_label_val)}')

# ax.set_xlabel("Longitude")
# ax.set_ylabel("Latitude")
# ax.grid(True)
# ax.legend()
# plt.show()

### Task 3 : Change the spatial resolution from 800m to 50km. This requires keeping grid # 1, counting to grid # 51 and removing grids between 1 and 51. Repeat the same procedure all the way towards the last grid, which is grid # ~700,000. After that should have nearly 280 grids.

In [ ]:
downsample_factor = 63

coarsened_labels = masked_labels.isel(
    latitude=slice(0, None, downsample_factor),
    longitude=slice(0, None, downsample_factor)
)

num_selected = int(coarsened_labels.notnull().sum().values)
print(f"Number of selected grids for Task 3: {num_selected}")

In [ ]:
selected_lons, selected_lats = np.meshgrid(
    coarsened_labels.longitude.values,
    coarsened_labels.latitude.values
)

valid_mask = ~np.isnan(coarsened_labels.values)

In [ ]:
#selected_lons, selected_lats = np.meshgrid(
#    coarsened_labels.longitude.values,
#    coarsened_labels.latitude.values
#)

#valid_mask = ~np.
# isnan(coarsened_labels.values)
#print(valid_mask)#print(coarsened_labels.values)

# 1. ساخت شبکه مختصات
selected_lons, selected_lats = np.meshgrid(
    coarsened_labels.longitude.values,
    coarsened_labels.latitude.values)

# 2. بررسی valid_mask
valid_mask = ~np.isnan(coarsened_labels.values)
print(f"تعداد مقادیر معتبر: {np.sum(valid_mask)}")

# 3. فیلتر کردن داده‌های معتبر
valid_lons = selected_lons[valid_mask]
valid_lats = selected_lats[valid_mask]

# 4. چاپ داده‌ها
print(valid_lons[:10])  # اولین ۱۰ مقدار از longitude‌های معتبر
print(valid_lats[:10])  # اولین ۱۰ مقدار از latitude‌های معتبر



In [ ]:
# fig, ax = plt.subplots(figsize=(12, 10))
# texas_shape.plot(ax=ax, facecolor='lightgray', edgecolor='black', alpha=0.8)
# ax.scatter(selected_lons[valid_mask], selected_lats[valid_mask], color='red', marker='o', s=35, label=f'{num_selected} Selected Grids')
# ax.set_xlabel("Longitude")
# ax.set_ylabel("Latitude")
# ax.grid(True)
# ax.legend()
# ax.set_aspect('equal', adjustable='box')
# plt.show()

### Task 4 : Precipitation data is hourly based. At all ~280 grids, sum precipitation data in each day to switch to daily-based data.

In [ ]:
labeled_ds = ds_texas.where(mask)
labeled_ds['grid_label'] = label_da.where(mask)
downsample_factor = 63

ds_50km = labeled_ds.isel(
    latitude=slice(0, None, downsample_factor),
    longitude=slice(0, None, downsample_factor)
)

ds_50km = ds_50km.where(ds_50km['grid_label'].notnull(), drop=True)

In [ ]:
ds_50km['APCP_surface']


In [ ]:
# === Task 3: Downsample AFTER clipping and masking ===

lat_vals = ds_texas.latitude.values
lon_vals = ds_texas.longitude.values

# Texas grid downsampled to ~50 km
selected_lats = lat_vals[::63]
selected_lons = lon_vals[::63]

selected_lons, selected_lats = np.meshgrid(selected_lons, selected_lats)

print("Downsampled grid shape:", selected_lats.shape)


In [ ]:
lon_test = selected_lons.flat[50]
lat_test = selected_lats.flat[50]

pd = ds_texas.sel(longitude=lon_test, latitude=lat_test, method='nearest')
print("Value:", pd['TMP_2maboveground'].isel(time=0).values)


In [ ]:
# Downsample the actual Texas dataset coordinates (1 km → ~50 km)
ds_lat = ds_texas.latitude.values
ds_lon = ds_texas.longitude.values

# stride = 63
lat_down = ds_lat[::63]
lon_down = ds_lon[::63]

# Build meshgrid
selected_lons, selected_lats = np.meshgrid(lon_down, lat_down)

print("Downsampled grid shape:", selected_lats.shape)


In [ ]:
lon_test = selected_lons.flat[50]
lat_test = selected_lats.flat[50]

value = ds_texas.sel(longitude=lon_test, latitude=lat_test, method='nearest')['TMP_2maboveground'].isel(time=0).values
print(value)


In [ ]:
test_lat = selected_lats[0, 0]   # first selected lat
print("test_lat:", test_lat)

print("Is there any matching latitude?",
      np.isclose(test_lat, ds_texas.latitude.values).any())


In [ ]:
test_lon = selected_lons[0, 0]
print("test_lon:", test_lon)

print("Is there any matching longitude?",
      np.isclose(test_lon, ds_texas.longitude.values).any())


In [ ]:
import numpy as np

tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
mask_valid = np.isfinite(tmp0)

print("Percentage of valid temperature values:",
      100 * np.mean(mask_valid), "%")


In [ ]:
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0)
valid_mask = np.isfinite(tmp0.values)


In [ ]:
valid_lat, valid_lon = np.where(valid_mask)

# pick every Nth valid cell
stride = len(valid_lat) // 280
lat_idx = valid_lat[::stride]
lon_idx = valid_lon[::stride]


In [ ]:
test = ds_texas['TMP_2maboveground'].isel(time=0,
                                          latitude=lat_idx[10],
                                          longitude=lon_idx[10]).values
print(test)


In [ ]:
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
valid_mask = np.isfinite(tmp0)
valid_lat_idx, valid_lon_idx = np.where(valid_mask)


In [ ]:
selected_lats = ds_texas.latitude.values[lat_idx]
selected_lons = ds_texas.longitude.values[lon_idx]


In [ ]:
test = ds_texas['TMP_2maboveground'].isel(
    time=0,
    latitude=lat_idx[0],
    longitude=lon_idx[0]
).values
print(test)


In [ ]:
target_points = 280
total_valid = len(valid_lat_idx)

indices = np.linspace(0, total_valid - 1, target_points).astype(int)

lat_idx = valid_lat_idx[indices]
lon_idx = valid_lon_idx[indices]


In [ ]:
ds_texas = ds_texas_bbox.rio.clip(texas_shape.geometry, drop=True)


In [ ]:
# ds_texas is already clipped to Texas exactly using rioxarray

# Compute valid mask from temperature field
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
valid_mask = np.isfinite(tmp0)

# Coordinates of valid gridcells
valid_lat_idx, valid_lon_idx = np.where(valid_mask)

# Compute stride to get ~280 points
target_points = 280
stride = max(1, len(valid_lat_idx) // target_points)

# Downsample
lat_idx = valid_lat_idx[::stride]
lon_idx = valid_lon_idx[::stride]

# Use indices to extract lat/lon
selected_lats = ds_texas.latitude.values[lat_idx]
selected_lons = ds_texas.longitude.values[lon_idx]

# Meshgrid
selected_lons, selected_lats = np.meshgrid(selected_lons, selected_lats)

print("Selected grid shape:", selected_lats.shape)


In [ ]:
test = ds_texas['TMP_2maboveground'].isel(
    time=0,
    latitude=lat_idx[5],
    longitude=lon_idx[5]
).values

print("Temperature at a selected point:", test)


In [ ]:
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
valid_mask = np.isfinite(tmp0)
valid_lat_idx, valid_lon_idx = np.where(valid_mask)


In [ ]:
target_points = 280
total_valid = len(valid_lat_idx)

indices = np.linspace(0, total_valid - 1, target_points).astype(int)

lat_idx = valid_lat_idx[indices]
lon_idx = valid_lon_idx[indices]


In [ ]:
selected_lats = ds_texas.latitude.values[lat_idx]
selected_lons = ds_texas.longitude.values[lon_idx]


In [ ]:
test = ds_texas['TMP_2maboveground'].isel(
    time=0,
    latitude=lat_idx[0],
    longitude=lon_idx[0]
).values
print(test)


In [ ]:
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
valid_pct = np.mean(np.isfinite(tmp0)) * 100
print("Valid temperature %:", valid_pct)


In [ ]:
tmp0 = ds_texas['TMP_2maboveground'].isel(time=0).values
valid_mask = np.isfinite(tmp0)

valid_lat_idx, valid_lon_idx = np.where(valid_mask)


In [ ]:
# lat_idx and lon_idx already computed from valid_mask
# These index pairs guarantee VALID temperature points
selected_latitudes = ds_texas.latitude.values[lat_idx]
selected_longitudes = ds_texas.longitude.values[lon_idx]

output_file = f"hourly_temperature_data_{Start_date}_{End_date}.csv"
batch_size = 10

with open(output_file, "w") as f:
    f.write("longitude,latitude,datetime,temperature_K\n")

total_records = 0

for batch_start in range(0, len(lat_idx), batch_size):
    batch_end = min(batch_start + batch_size, len(lat_idx))
    
    print(f"Processing batch: {batch_start} to {batch_end-1}")
    
    batch_records = 0
    
    for k in range(batch_start, batch_end):
        
        i = lat_idx[k]   # row index
        j = lon_idx[k]   # column index
        
        lat_val = ds_texas.latitude.values[i]
        lon_val = ds_texas.longitude.values[j]
        
        # extract full hourly timeseries using index-based selection
        cell = ds_texas['TMP_2maboveground'].isel(latitude=i, longitude=j)
        
        # convert to pandas Series
        series = cell.to_series()
        
        # write valid temperatures to file
        with open(output_file, "a") as f:
            for dt, temp in series.items():
                if np.isnan(temp):
                    continue
                f.write(f"{lon_val},{lat_val},{dt},{temp}\n")
                batch_records += 1

        print(f"  Point #{k}: {batch_records} records added")
    
    total_records += batch_records
    print(f"Batch complete: {batch_records} records (Total: {total_records})")

print("\nDone! Temperature data saved to", output_file)


### Resume

In [ ]:
# ====== RESUME FROM POINT 154 ======
resume_idx = 154  # <-- Change this number to resume from another point

output_file = f"hourly_temperature_data_{Start_date}_{End_date}.csv"
batch_size = 9

# IMPORTANT:
# Do NOT overwrite the file — append instead.
# So open in append mode except for the header.
# Remove the "write header" part.

total_records = 0

for batch_start in range(resume_idx, len(lat_idx), batch_size):
    batch_end = min(batch_start + batch_size, len(lat_idx))
    
    print(f"Processing batch: {batch_start} to {batch_end-1}")
    
    batch_records = 0
    
    for k in range(batch_start, batch_end):
        
        # Skip points already processed last time
        if k < resume_idx:
            continue
        
        i = lat_idx[k]
        j = lon_idx[k]

        lat_val = ds_texas.latitude.values[i]
        lon_val = ds_texas.longitude.values[j]
        
        cell = ds_texas['TMP_2maboveground'].isel(latitude=i, longitude=j)
        series = cell.to_series()

        # Append new data
        with open(output_file, "a") as f:
            for dt, temp in series.items():
                if np.isnan(temp):
                    continue
                f.write(f"{lon_val},{lat_val},{dt},{temp}\n")
                batch_records += 1
        
        print(f"  Point #{k}: {batch_records} records added")
    
    total_records += batch_records
    print(f"Batch complete: {batch_records} records (Total: {total_records})")

print("\nResume complete. New data appended to", output_file)
